<a href="https://colab.research.google.com/github/abolfazlshahsavaryyy/DT/blob/main/DT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [106]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier

In [107]:
def compute_gini(*p):
    if not (0.999 <= sum(p) <= 1.001):
        raise ValueError("The probabilities must sum to 1.")

    result = 1
    for i in p:
        result -= i**2

    return result

In [108]:
def compute_entropy(*p):
    if not (0.999 <= sum(p) <= 1.001):
        raise ValueError("The probabilities must sum to 1.")

    result = 0
    for i in p:
        if i > 0:
            result += i * np.log2(i)

    return -result

In [109]:
def load_dataset(url:str)->pd.DataFrame:
    raw_df=pd.read_csv(url)
    return raw_df


In [110]:
def dropnan_df(raw_df:pd.DataFrame)->pd.DataFrame:
    raw_df=raw_df.drop("id",axis=1)
    raw_df=raw_df.drop("row",axis=1)
    return raw_df.dropna()


In [111]:
def count_values_columns(df:pd.DataFrame,cols:list):
    for s in cols:
        print(df[s].value_counts())


In [112]:
def Classify_continuous(df:pd.DataFrame,col_name ):

    col=df[col_name]
    min=np.min(col)
    max=np.max(col)
    mean=np.mean([max,min])
    mean_75=np.mean([max,mean])
    mean_25=np.mean([min,mean])
    df.loc[df[col_name]<=min,col_name]=0
    df.loc[(df[col_name]>min )& (df[col_name]<=mean_25),col_name]=1
    df.loc[(df[col_name]>mean_25) & (df[col_name]<=mean),col_name]=2
    df.loc[(df[col_name]>mean) & (df[col_name]<=mean_75),col_name]=3
    df.loc[(df[col_name]>mean_75) & (df[col_name]<=max),col_name]=4
    df.loc[df[col_name]>max ,col_name]=5


In [113]:
def custome_classify_age(df:pd.DataFrame,col="Age")->pd.DataFrame:
    df.loc[(df[col]>=1 )&(df[col]<=6),col]=0
    df.loc[(df[col]==7 ),col]=1
    df.loc[(df[col]==8 ),col]=2
    df.loc[(df[col]==9 ),col]=3
    df.loc[(df[col]==10 ),col]=4
    df.loc[(df[col]==11 ),col]=5
    #over 60 years
    df.loc[(df[col]>=12 )&(df[col]<=13),col]=6



In [114]:
def custome_classify_bmi(df:pd.DataFrame,col="BMI"):
    df.loc[(df[col]<=18.5),col]=0
    df.loc[(df[col]>18.5)&(df[col]<=25),col]=1
    df.loc[(df[col]>25)&(df[col]<=30),col]=2
    df.loc[(df[col]>30)&(df[col]<=35),col]=3
    df.loc[(df[col]>35)&(df[col]<=40),col]=4
    df.loc[(df[col]>40),col]=5
    return df

In [115]:
def custome_classify_PhysHlth_MentHlth(df:pd.DataFrame):
    df.loc[(df["MentHlth"]<6),"MentHlth"]=0
    df.loc[(df["MentHlth"]>=6),"MentHlth"]=1

    df.loc[(df["PhysHlth"]<6),"PhysHlth"]=0
    df.loc[(df["PhysHlth"]>=6),"PhysHlth"]=1



In [116]:
def classify_df(df:pd.DataFrame)-> pd.DataFrame:
    gender_map={
        "Female":0,
        "Male":1
    }
    type_travel_map={
        "Business travel":0,
        "Personal Travel":1
    }
    customer_type_map={
        "Loyal Customer":0,
        "disloyal Customer":1
    }
    class_map={
        "Business":0,
        "Eco":1,
        "Eco Plus":2
    }
    df["Gender"]=df["Gender"].map(gender_map)
    df["Type of Travel"]=df["Type of Travel"].map(type_travel_map)
    df["Customer Type"]=df["Customer Type"].map(customer_type_map)
    df["Class"]=df["Class"].map(class_map)


    return df

In [117]:
def split_train_test_valid(df, random_state=42, test_size=0.2, valid_size=0.2):
    n = len(df)
    np.random.seed(random_state)
    indexes = np.random.permutation(n)


    test_size_df = int(n * test_size)
    valid_size_df = int((n - test_size_df) * valid_size)
    train_size_df = n - (test_size_df + valid_size_df)


    df_train = df.iloc[indexes[:train_size_df]].reset_index(drop=True)
    df_test = df.iloc[indexes[train_size_df:train_size_df + test_size_df]].reset_index(drop=True)
    df_valid = df.iloc[indexes[train_size_df + test_size_df:]].reset_index(drop=True)

    return df_train, df_test, df_valid

In [118]:
def split_X_y(df,y_label="class"):
    y=df[y_label]
    X=df.drop(columns=[y_label])
    return X,y

In [119]:
url_dataset="diabetes_data.csv"
df=load_dataset(url_dataset)
custome_classify_age(df,"Age")
custome_classify_bmi(df,"BMI")
custome_classify_PhysHlth_MentHlth(df)
train_df,test_df,valid_df=split_train_test_valid(df,random_state=42,test_size=0.2,valid_size=0.2)
X_train,y_train=split_X_y(train_df,"Diabetes")
X_test,y_test=split_X_y(test_df,"Diabetes")
X_valid,y_valid=split_X_y(valid_df,"Diabetes")
